In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-07,France,139158.0,0.0,26137.0,0.0,0.0,0.0
0,2020-05-07,China,84053.0,0.0,4662.0,0.0,0.0,0.0
0,2020-05-07,Italy,215534.0,0.0,29923.0,0.0,0.0,0.0
0,2020-05-07,Spain,221508.0,0.0,26021.0,0.0,0.0,0.0
0,2020-05-07,United States,1253521.0,0.0,75713.0,0.0,0.0,0.0
0,2020-05-07,World,3799025.0,0.0,269575.0,0.0,0.0,0.0
0,2020-05-07,United Kingdom,206110.0,0.0,30756.0,0.0,0.0,0.0
0,2020-05-07,Germany,166938.0,0.0,7273.0,0.0,0.0,0.0
0,2020-05-07,Iran,103019.0,0.0,6489.0,0.0,0.0,0.0
0,2020-05-07,Turkey,133414.0,0.0,3634.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-08,France,139158.0,137779.0,26137.0,25987.0,1379.0,150.0
0,2020-05-08,China,84053.0,83976.0,4662.0,4637.0,77.0,25.0
0,2020-05-08,Italy,215534.0,215858.0,29923.0,29958.0,-324.0,-35.0
0,2020-05-08,Spain,221508.0,221447.0,26021.0,26070.0,61.0,-49.0
0,2020-05-08,United States,1253521.0,1256972.0,75713.0,75670.0,-3451.0,43.0
0,2020-05-08,World,3799025.0,3807852.0,269575.0,269068.0,-8827.0,507.0
0,2020-05-08,United Kingdom,206110.0,206715.0,30756.0,30615.0,-605.0,141.0
0,2020-05-08,Germany,166938.0,167300.0,7273.0,7266.0,-362.0,7.0
0,2020-05-08,Iran,103019.0,103135.0,6489.0,6486.0,-116.0,3.0
0,2020-05-08,Turkey,133414.0,133721.0,3634.0,3641.0,-307.0,-7.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")